# RAG

Имя, Фамилия:



---

## Часть 1. 6 баллов

Нужно взять текущий датасет или любой другой, понравившийся вам в формате question/answer. Не рекомендуется брать датасет больше текущего.
Далее необходимо реализовать все функции ниже и сделать всё то, что было на лекции-семинаре. Построить RAG на 2 разных вариантах индекса,
замерить качество, сделать вывод о том, какой из вариантов лучше. Качество финальной реализации RAG не обязательно оценивать на большом объеме, достаточно несколько десятков примеров (например до 100), если каждая генерация LLM будет либо слишком долгой или вы ограничены небольшим кол-вом бесплатных токенов (в случае gigachat api).

В качестве LLM можно взять open-source сервис или воспользоваться бесплатной квотой в 900_000 [gigachat](https://developers.sber.ru/studio). Функция похода в api гигачат есть в gigachat.py,
где нужно лишь добавить свой токен

#### Поднять elasticsearch локально
```
docker run -p 9200:9200 --rm --env-file elastic.env docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```
содержимое elastic.env
```
discovery.type=single-node
xpack.security.enabled=false
```
#### Поднять LLM локально
```
docker run --rm -p 11434:11434 -v ollama:/root/.ollama --name ollama ollama/ollama:0.5.12

# в отдельном экране скачиваем нужную модель
docker exec -it ollama bash
ollama pull <выбранная модель> # выбрать модель https://github.com/ollama/ollama
```

## Часть 2. 4 балла

На занятии мы уже обсуждали, что один из возможных способов улучшить knn поиск - это улучшить векторный енкодер. 
В задании предлагается ровно это и сделать, взять нужный датасет и дообучить его, не забыв про полный цикл задачи
машинного обучения (реализовать все необходиомое для обучения, поделить выборки на train/val/test, замерить качество и др.).
По итогу качество приближенного поиска должно стать лучше предобученной модели. Базовую модель, технологию и способ дообучения вы выбирайте сами.

## (Опционально) Часть 3. дополнительные 5 баллов

Сверхнормы предлагается 2 опциональных задания:
- (4 балла) сделать гибридный поиск (полнотекстовый поиск + приближенный), многие современные поисковые системы (например [baidu](https://arxiv.org/abs/2106.03373)) используют именно такой подход. В идеале это должно работать за 1 поход
в поисковый сервис
- (1 балл) сделать UI интерфейс к вашей RAG-системе. Подсказка: посмотрите в сторону streamlit. 
Основная функциональность: ввод текста в специальной строке ввода и ответ RAG-сервиса на одном экране

## Домашнее задание № 11

Выполните задания, описанные выше:

+ Мягкий дедлайн: `19.04.25 23:59`
+ Жесткий дедлайн: `26.04.25 23:59` (половина баллов)


После жесткого дедлайна задание не принимается.

---

In [ ]:
from datasets import load_dataset
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import tqdm
import pandas as pd
import re
import numpy as np
from openai import OpenAI
import os
import requests
import json
from dotenv import load_dotenv
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

load_dotenv()

from gigachat import get_gigachat_response

# %load_ext autoreload
# %autoreload 2

# RAG intro

In [ ]:
# llama3.2:1b - 1В
# deepseek-r1 - 7В

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [ ]:
def get_response_any_llm(prompt, client, model_name):
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


def build_english_prompt(query, search_results):
    prompt_template = """
You will get corpus of texts and one question. Your aim is finding an exact answer using the given context. Try to give short answer to the question

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for i, text in enumerate(search_results):
        context = context + f"text_number: {i} text: {text}\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
get_response_any_llm("что такое кошка?", client, "llama3.2:1b")

In [ ]:
get_response_any_llm("расскажи о китах", client, "llama3.2:1b")

In [ ]:
get_response_any_llm("what is a cat?", client, "llama3.2:1b")

In [ ]:
question = "who won the match sri lanka or india"
context = "India won the Test series 1–0, after the first and third matches were drawn.[15] India won the ODI series 2–1, their eighth consecutive series win since beating Zimbabwe in June 2016.[16] India won the T20I series 3–0.[17]"
prompt = build_english_prompt(question, [context])
print(prompt)

In [ ]:
get_response_any_llm(prompt, client, "llama3.2:1b")

In [ ]:
get_gigachat_response(prompt)

# Search engine

In [ ]:
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

In [ ]:
ds = load_dataset("kuznetsoffandrey/sberquad")
validation = ds['validation']

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "context": {"type": "text"}
        }
    }
}

simple_index_name = "sberquad-index"

es_client.indices.delete(index=simple_index_name, ignore_unavailable=True)
es_client.indices.create(index=simple_index_name, body=index_settings)

In [ ]:
for doc in tqdm.tqdm(validation):
    document = {"context": doc['context']}
    try:
        es_client.index(index=simple_index_name, document=document, id=doc['id'])
    except Exception as e:
        print(e)

In [ ]:
# реализовать функцию запроса в elasticsearch для получения ответа с помощью полнотекстового поиска
def search(question, index_name, k):
    """
    question - вопрос
    index_name - индекс, в который осуществляется поход
    k - кол-во контекстов, которое должно вернуться

    Output: набор контекстов, ответ поиска

    Подсказка:
    query = {
        "match": {
            "context": question
        }
    }
    response = es_client.search(index=index_name, query=query)
    """
    pass

# simple rag

In [ ]:
def build_russian_prompt(query, search_results):
    prompt_template = """
Тебе будет дан набор текстов и один вопрос. Твоя задача найти точный ответ на на вопрос, имея заданный контекст. Старайся давать короткий ответ на вопрос

ВОПРОС: {question}

КОНТЕКСТ: 
{context}
""".strip()

    context = ""
    
    for i, text in enumerate(search_results):
        context = context + f"text_number: {i} text: {text}"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
# реализовать функцию простого rag, использующий полнотекстовый поиск
def simple_rag(query, top_n_contexts):
    pass

obj = validation[4500]
simple_rag(obj['question'], 5)

# simple evaluate

In [5]:
# реализовать функцию оценки качества retrieve стадии
def precision_at_k(dataset, k, search_func):
    """
    Функция для замера качества поиска с помощью elasticsearch
    
    dataset - датасет для замера
    k - топ на котором замеряется качество
    search_func - функция поиска в elasticsearch
    """
    pass

# Improved knn-search

In [ ]:
# model = SentenceTransformer('sentence-transformers/paraphrase-TinyBERT-L6-v2')
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embedding = model.encode(validation[0]["question"])
print(len(embedding))

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "context": {"type": "text"},
            "id": {"type": "integer"},
            "context_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

knn_index_name = "sberquad-index-knn"

es_client.indices.delete(index=knn_index_name, ignore_unavailable=True)
es_client.indices.create(index=knn_index_name, body=index_settings)

In [ ]:
for i, doc in tqdm.tqdm(enumerate(validation), total=len(validation)):
    doc_embed = model.encode(doc["context"]).tolist()
    doc = {"context": doc['context'], "id": doc['id'], "context_vector": doc_embed}
    try:
        es_client.index(index=knn_index_name, document=doc)
    except Exception as e:
        print(e)

In [ ]:
# реализовать функцию запроса в elasticsearch для получения ответа с помощью приближенного поиска
def knn_search(question, encoder, index_name, k):
    """
    question - вопрос
    encoder - енкодер для векторизации запроса
    index_name - индекс, в который осуществляется поход
    k - кол-во контекстов, которое должно вернуться

    Output: набор контекстов, ответ поиска

    Подсказка:
    query = {
        "field": "context_vector",
        "query_vector": embedding,
        "k": k,
        "num_candidates": 10
    }
    response = es_client.knn_search(index=knn_index_name, knn=query)
    """
    pass


# реализовать функцию rag, использующий knn поиск
def knn_rag(question, top_n_contexts):
    pass

# Evaluate end-2-end RAG

In [ ]:
def cosine_quality(dataset, encoder, search_func):
    """
    dataset - датасет
    encoder - енкодер для векторизации запроса
    search_func - функция поиска в elasticsearch
    """
    pass